In [1]:
import numpy as np
import tensorflow as tf
import keras.backend as K
import gym
from critic import Critic
from actor import Actor

import numpy as np
import tensorflow as tf
import keras.backend as K

from keras.initializers import RandomUniform
from keras.models import Model, load_model
from keras.layers import Input, Dense, Reshape, LSTM, Lambda, BatchNormalization, GaussianNoise, Flatten
from keras.optimizers import Adam 


#Environment
env = gym.make('MountainCarContinuous-v0')
env.seed(1)  # for comparison
env_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]


#Agent
lr,gamma = 0.001, 0.99
act_range = 10
#actor = Actor(env_dim, act_dim, lr)
critic = Critic(env_dim, act_dim, lr, gamma)


inp = Input(shape = (env_dim,))
x = Dense(256, activation='relu')(inp)
#x = GaussianNoise(1.0)(x)
#x = Flatten()(x)   # I assume this is if the input is a convolutional neural net?
x = Dense(128, activation='relu')(x)
#x = GaussianNoise(1.0)(x)
out = Dense(act_dim, activation='tanh', kernel_initializer=RandomUniform())(x)
out = Lambda(lambda i: i * act_range)(out)
model = Model(inp, out)
model.compile(loss='mse',optimizer=Adam(lr))

/home/kokeeffe/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.


In [1]:
""" Define function  """
#Inputs
state_pl = model.input
action_grads_pl = K.placeholder(shape=(None,1))  

#Find grad_(pars) mu(state)
mu_pl = model.output
pars = model.trainable_weights
grads = tf.gradients(mu_pl, pars, - action_grads_pl)
grads_and_pars = zip(grads, model.trainable_weights)  #keras needs this format 
operation = tf.train.AdamOptimizer(lr).apply_gradients(grads_and_pars)
func = K.function(inputs = [state_pl, action_grads_pl], outputs = [operation])

#grads_and_pars_pl = [ (tf.placeholder("float",shape=x[0].shape), x[1]) for x in grads_and_pars]
#updates = tf.train.AdamOptimizer(lr).apply_gradients(grads_and_pars_pl)
#func = K.function(inputs = [state_pl, action_grads_pl, grads_and_pars_pl], outputs = [updates])

func2 = K.function([state_pl, action_grads_pl],[operation])

""" Test function  """
state = np.array([[1,2]])
action = np.array([[0.5]])
mu = model.predict(state)
pars = model.trainable_weights
action_grads = critic.find_action_grads([state,action])[0]
#func([state,action_grads, grads_and_pars])
func2([state, action_grads])

NameError: name 'model' is not defined

### Pure  tensorflow 

In [200]:
#Define model
inp = Input(shape = (env_dim,))
x = Dense(3, activation='relu')(inp)
#x = GaussianNoise(1.0)(x)
#x = Flatten()(x)   # I assume this is if the input is a convolutional neural net?
x = Dense(3, activation='relu')(x)
#x = GaussianNoise(1.0)(x)
out = Dense(act_dim, activation='tanh', kernel_initializer=RandomUniform())(x)
out = Lambda(lambda i: i * act_range)(out)
model = Model(inp, out)
model.compile(loss='mse',optimizer=Adam(lr))


#Define the graph
mu_pl = model.output
action_pl = K.placeholder(shape=(None,act_dim),name='action_pl')
pars = model.trainable_weights
grads = tf.gradients(mu_pl, pars, -action_pl)
grads_and_pars = zip(grads,pars)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
operation = optimizer.apply_gradients(grads_and_pars)

#Do the operation
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())

state = np.array([[1.2,1.4]])
action = np.array([[3.4]])
print model.predict(state)
action_grads = critic.find_action_grads([state,action])[0]

sess.run(operation, feed_dict={inp:state, action_pl:action_grads})
print '\n ' + str(model.predict(state))

[[-0.00171607]]

 [[0.02899274]]


Great! Now to figure out how to get keras to do it

### Keras

In [2]:
#Define model
inp = Input(shape = (env_dim,))
x = Dense(3, activation='relu')(inp)
#x = GaussianNoise(1.0)(x)
#x = Flatten()(x)   # I assume this is if the input is a convolutional neural net?
x = Dense(3, activation='relu')(x)
#x = GaussianNoise(1.0)(x)
out = Dense(act_dim, activation='tanh', kernel_initializer=RandomUniform())(x)
out = Lambda(lambda i: i * act_range)(out)
model = Model(inp, out)
model.compile(loss='mse',optimizer=Adam(lr))


#Define the graph
state_pl = model.input
mu_pl = model.output
action_pl = K.placeholder(shape=(None,act_dim),name='action_pl')
pars = model.trainable_weights
grads = tf.gradients(mu_pl, pars, -action_pl)
grads_and_pars = zip(grads,pars)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
operation = optimizer.apply_gradients(grads_and_pars, name='opt')
apply_grad = K.function([state_pl, action_grads_pl],[operation])


state = np.array([[1.2,1.4]])
action = np.array([[3.4]])
print model.predict(state)
action_grads = critic.find_action_grads([state,action])[0]

sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
apply_grad([state,action_grads])

#sess.run(operation, feed_dict={inp:state, action_pl:action_grads})
#print '\n ' + str(model.predict(state))

NameError: name 'action_grads_pl' is not defined

### Keras for A2C

In [9]:
import numpy as np
import tensorflow as tf
import keras.backend as K
import gym
from critic import Critic
from actor import Actor

import numpy as np
import tensorflow as tf
import keras.backend as K

from keras.initializers import RandomUniform
from keras.models import Model, load_model
from keras.layers import Input, Dense, Reshape, LSTM, Lambda, BatchNormalization, GaussianNoise, Flatten
from keras.optimizers import Adam 


#Environment
env = gym.make('MountainCarContinuous-v0')
env.seed(1)  # for comparison
env_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]


#Agent
lr,gamma = 0.001, 0.99
act_range = 10
#actor = Actor(env_dim, act_dim, lr)
critic = Critic(env_dim, act_dim, lr, gamma)


inp = Input(shape = (env_dim,))
x = Dense(256, activation='relu')(inp)
#x = GaussianNoise(1.0)(x)
#x = Flatten()(x)   # I assume this is if the input is a convolutional neural net?
x = Dense(128, activation='relu')(x)
#x = GaussianNoise(1.0)(x)
out = Dense(act_dim, activation='tanh', kernel_initializer=RandomUniform())(x)
out = Lambda(lambda i: i * act_range)(out)
model = Model(inp, out)
model.compile(loss='mse',optimizer=Adam(lr))



#Define model
inp = Input(shape = (env_dim,))
x = Dense(3, activation='relu')(inp)
#x = GaussianNoise(1.0)(x)
#x = Flatten()(x)   # I assume this is if the input is a convolutional neural net?
x = Dense(3, activation='relu')(x)
#x = GaussianNoise(1.0)(x)
out = Dense(act_dim, activation='tanh', kernel_initializer=RandomUniform())(x)
out = Lambda(lambda i: i * act_range)(out)
model = Model(inp, out)
model.compile(loss='mse',optimizer=Adam(lr))


#Define the graph
state_pl = model.input
mu_pl = model.output
action_grad_pl = K.placeholder(shape=(None,act_dim),name='action_pl')
pars = model.trainable_weights
grads = tf.gradients(mu_pl, pars, -action_grad_pl)

#Do updates
loss = grads
opt = Adam(lr)
do_updates = opt.get_updates(loss=loss,params=pars,grads=grads)  #this is an operation
apply_grad = K.function([state_pl, action_grad_pl], [], updates = do_updates)

#Trial session
state = np.array([[1.2,1.4]])
action = np.array([[3.4]])
print model.predict(state)
action_grads = critic.find_action_grads([state,action])[0]
apply_grad([state,action_grads])
print model.predict(state)

Using TensorFlow backend.


TypeError: get_updates() got an unexpected keyword argument 'grads'

### Change keras source code

In [2]:
np.random.seed(0)


#Define model
inp = Input(shape = (env_dim,))
x = Dense(3, activation='relu')(inp)
#x = GaussianNoise(1.0)(x)
#x = Flatten()(x)   # I assume this is if the input is a convolutional neural net?
x = Dense(3, activation='relu')(x)
#x = GaussianNoise(1.0)(x)
out = Dense(act_dim, activation='tanh', kernel_initializer=RandomUniform())(x)
out = Lambda(lambda i: i * act_range)(out)
model = Model(inp, out)
model.compile(loss='mse',optimizer=Adam(lr))

#Place holders (think of these as function inputs)
state_placeholder = model.input
action_placeholder =  model.output
grad_actions_pl = K.placeholder(shape=(None,act_dim))

#Find log-probs
log_action = K.log(action_placeholder) 

#I want to the gradient to by on the log(pi) only, so 'protect' the advantage
# This is the way keras works -- I think!
#eligibility = log_action*K.stop_gradient(grad_actions_pl)
eligibility = log_action*K.stop_gradient(grad_actions_pl)


loss = K.mean(eligibility)

#Define optimizer
adam = Adam()
pars = model.trainable_weights
updates = adam.get_updates(params=pars,loss=loss)

#grads_in = tf.gradients(loss,pars)
#updates, grads_out = adam.get_updates(params=pars,loss=loss, grads = grads_in)

#Then return
func = K.function([state_placeholder, grad_actions_pl],[],updates=
                 updates)



state = np.array([[1.2,1.4]])
action = np.array([[3.4]])
print model.predict(state)
action_grads = critic.find_action_grads([state,action])[0]
func([state,action_grads])
print model.predict(state)

[<tf.Tensor 'gradients_2/dense_13/MatMul_grad/MatMul_1:0' shape=(2, 3) dtype=float32>, <tf.Tensor 'gradients_2/dense_13/BiasAdd_grad/BiasAddGrad:0' shape=(3,) dtype=float32>, <tf.Tensor 'gradients_2/dense_14/MatMul_grad/MatMul_1:0' shape=(3, 3) dtype=float32>, <tf.Tensor 'gradients_2/dense_14/BiasAdd_grad/BiasAddGrad:0' shape=(3,) dtype=float32>, <tf.Tensor 'gradients_2/dense_15/MatMul_grad/MatMul_1:0' shape=(3, 1) dtype=float32>, <tf.Tensor 'gradients_2/dense_15/BiasAdd_grad/BiasAddGrad:0' shape=(1,) dtype=float32>]
Check 3
[<tf.Tensor 'AssignAdd_1:0' shape=() dtype=int64_ref>, <tf.Tensor 'Assign_24:0' shape=(2, 3) dtype=float32_ref>, <tf.Tensor 'Assign_25:0' shape=(2, 3) dtype=float32_ref>, <tf.Tensor 'Assign_26:0' shape=(2, 3) dtype=float32_ref>, <tf.Tensor 'Assign_27:0' shape=(3,) dtype=float32_ref>, <tf.Tensor 'Assign_28:0' shape=(3,) dtype=float32_ref>, <tf.Tensor 'Assign_29:0' shape=(3,) dtype=float32_ref>, <tf.Tensor 'Assign_30:0' shape=(3, 3) dtype=float32_ref>, <tf.Tensor 'As

In [37]:
grads

[<tf.Tensor 'gradients_26/dense_49/MatMul_grad/MatMul_1:0' shape=(2, 3) dtype=float32>,
 <tf.Tensor 'gradients_26/dense_49/BiasAdd_grad/BiasAddGrad:0' shape=(3,) dtype=float32>,
 <tf.Tensor 'gradients_26/dense_50/MatMul_grad/MatMul_1:0' shape=(3, 3) dtype=float32>,
 <tf.Tensor 'gradients_26/dense_50/BiasAdd_grad/BiasAddGrad:0' shape=(3,) dtype=float32>,
 <tf.Tensor 'gradients_26/dense_51/MatMul_grad/MatMul_1:0' shape=(3, 1) dtype=float32>,
 <tf.Tensor 'gradients_26/dense_51/BiasAdd_grad/BiasAddGrad:0' shape=(1,) dtype=float32>]

Sweet! It works.

In [3]:
gammas = [0.1,0.5,0.99]
lrs = [0.0001, 0.001, 0.01, 0.1, 1]
taus = [0.0001, 0.001, 0.01, 0.1, 1]


In [7]:
pars = [(g,lr) for g in gammas for lr in lrs]

[(0.1, 0.0001),
 (0.1, 0.001),
 (0.1, 0.01),
 (0.1, 0.1),
 (0.1, 1),
 (0.5, 0.0001),
 (0.5, 0.001),
 (0.5, 0.01),
 (0.5, 0.1),
 (0.5, 1),
 (0.99, 0.0001),
 (0.99, 0.001),
 (0.99, 0.01),
 (0.99, 0.1),
 (0.99, 1)]

In [11]:
l = ['1','2','3']
np.savetxt('temp.txt',l,fmt='%s')